# Apresentação

Neste notebook, é apresentado a construção de um gerador de sequência e o treinamento do modelo CNN. O gerador de sequência é útil para treinamento de modelos com grandes conjuntos de dados e uma das vantagens de utiliza-lo é a redução de memória ao carregar os dados, evitando que estoure, pois ao invés de carregar tudo de uma vez, apenas carrega em lotes ou "batches". Abaixo há mais detalhes a respeito da Implementação.

Obs: o conjunto de dados utilizado é o mesmo conjunto disponibilizado no drive da turma durante a aula.


## Sumário
- Carregamento do conjunto de dados com Google Drive
- Importação de Bibliotecas
- Implementação e execução do gerador de sequências
- Implementação do modelo de rede neural convolucional(CNN)
- Compilação
- Treinamento

### Carregamento do conjunto de dados com Google Drive

Primeiramente fazemos o quesito padrão e essencial para fazer o modelo funcionar, ou seja, é carregar o conjunto de dados que vamos utilizar durante todo o processo. Por isso, conectamos com o Google Drive para posteriormente indicar o diretório onde as imagens utilizadas e classificadas por categoria se encontram.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importação de bibliotecas

Após fazer a conexão com o Google Drive, é necessário fazer as devidas importações ou bibliotecas que serão utilizadas durante o processo de rodagem do modelo.

In [ ]:
import tensorflow as tf  # Biblioteca usada para criar e treinar redes neurais
from keras.utils import to_categorical, Sequence  # Importa funções utilitárias
from glob import glob  # Importa a função 'glob' para encontrar todos os caminhos que correspondem a um padrão
import numpy as np  # Importa a biblioteca numpy para manipulação de arrays
from keras.models import Sequential  # Importa o modelo 'Sequential' do Keras, uma pilha linear de camadas
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout  # Importa várias camadas do Keras usadas em redes neurais
from skimage.color import rgb2gray  # Importa função para converter imagens RGB em escala de cinza
from tensorflow.keras.models import Sequential  # Importa novamente o modelo 'Sequential', desta vez diretamente do tensorflow.keras
from tensorflow.keras.layers import DepthwiseConv2D, Conv2D  # Importa camadas de convolução, incluindo a DepthwiseConv2D, do tensorflow.keras
from tensorflow.keras.optimizers import Adam  # Importa o otimizador Adam do tensorflow.keras


### Implementação e execução do gerador de sequências

Inicia-se atribuindo uma classe específica apenas para o gerador de sequência denominado "GeradorSequencias".

- Definindo o "__ init __", para a inicialização, inserimos:

1. A entrada que listará o caminho do arquivo;
2. A saída que recebe saídas, ou seja, lista os identificadores ou labels para cada entrada;
3. Definimos o "batch_size" ou tamanho do lote que é o número de amostras por lote, o tamanho do lote é responsável por determinar quantas entradas e saídas são processadas durante o treino.

Após inserir o "__ init __", partimos para o cálculo do número de lotes representado pela função "__ len __".

- Função "__ len __":
Essa função é responsável por determinar quantos lotes podem ser formados com o conjunto de dados total a partir da divisão do número total de entradas pelo tamanho do lote. Conforme estabelece os cálculos, também determina quantas vezes o "get item" a seguir precisa ser chamado por época.

- Uso do ""__ getitem __":
Este é chamado pelo Keras durante o treino e obtém o lote específico de acordo com o índice.

- Cálculo de índices:
Determina os índices das amostras que serão incluídas no lote que funciona com duas direções o qual é esquerda e direita, a esquerda é calculada como o índice de início do lote, e direita é o índice final, que não ultrapassa o número total de entradas.

- Seleção de lote: As entradas e saídas para o lote são selecionadas usando os índices calculados. Essas amostras são as que serão carregadas e processadas.

- Processamento de entradas x, y: o x carrega as imagens pelos caminhos em batch_entradas, converte para um array np e normaliza os valores de pixel para 0 e 1 e divide por 255. Já o y converte os rótulos especificados em batch_saidas para representações categóricas usando to_categorical. O número 14 indica que existem 14 classes possíveis.

Durante o treinamento, o Keras chama o método __getitem__ repetidamente para obter cada lote e usa esses dados para treinar o modelo. O uso de Sequence garante que o modelo possa ser treinado em múltiplas threads e que a ordem dos dados seja mantida.


Obs: tal código foi feito juntamente com o professor durante a instrução

In [ ]:
class GeradorSequencias(Sequence):

    def __init__(self, entradas, saidas, batch_size):
        self.entradas = entradas  # Armazena os caminhos das imagens que serão usadas como entradas.
        self.saidas = saidas  # Armazena os caminhos das imagens ou labels que serão usadas como saídas (target).
        self.batch_size = batch_size  # Define o tamanho do lote (quantidade de amostras por lote).

    def __len__(self):
        # Define o número total de lotes por época.
        return len(self.entradas) // self.batch_size  # Calcula quantos lotes completos existem no dataset.

    def __getitem__(self, id):
        # Retorna um lote completo de entradas e saídas, dado um índice de lote específico.
        esquerda = id * self.batch_size  # Calcula o índice inicial do lote.
        direita = min(esquerda + self.batch_size, len(self.entradas))  # Calcula o índice final do lote; evita ultrapassar o tamanho da lista.
        batch_entradas = self.entradas[esquerda:direita]  # Seleciona o subconjunto das entradas para o lote atual.
        batch_saidas = self.saidas[esquerda:direita]  # Seleciona o subconjunto das saídas para o lote atual.

        x = np.array([img.imread(caminho)/255 for caminho in batch_entradas])  # Carrega as imagens das entradas, normaliza e converte para um array numpy.
        y = np.array([to_categorical(int(caminho.split('/')[-2]), 14) for caminho in batch_saidas])  # Processa as saídas para formato categórico.

        return x, y  # Retorna o lote de entradas e saídas processadas.


Abaixo há o caminho onde recebe todas as imagens presentes nos diretórios e subdiretórios.

In [ ]:
caminhos = glob('/content/drive/MyDrive/Aula08052024/imagens/*/*.jpg')

Embaralhamento aleatório dos elementos da lista "caminhos" para evitar viés de ordem.

In [ ]:
np.random.shuffle(caminhos) # embaralha aleatoriamente a lista 'caminhos' para garantir que a ordem dos dados seja aleatória antes de iniciar o treinamento de um modelo,
# o que pode ajuda na generalização do modelo
len(caminhos)

1400

Cria uma instância da classe GeradorSequencias com três argumentos "caminho", "caminhos" e "5". O 5 representa que cada batch de dados conterá 5 imagens(cinco entradas e cinco saída correspondentes)

In [ ]:
# Esta linha cria uma instância da classe 'GeradorSequencias', usando 'caminhos' tanto para as entradas quanto para as saídas, com um tamanho de lote de 5.
# Cada lote gerado pela sequência conterá 5 elementos
sequencia = GeradorSequencias(caminhos, caminhos, 5)

Ao atribuir o "sequencia.__len__()", será retornado o número total de batches. Esses batches é o qe o gerador cria e é calculado dividindo o número total de entradas pelo "batch_size"


OBs: como a divisão é inteira, caso houver entradas que não completam um batch inteiro, elas não são contabilizadas.

In [ ]:
sequencia.__len__()

280

Nesta célula, o código calcula o índice do primeiro item do batch a esquerda e o índice do item logo após o último item do batch que é a direita.

Dentro do método, o x normaliza as imagens(divide os valores dos pixels por 255 para fazer a conversão dos valores para o intervalo 0 e 1, enquanto y converte as saídas para vetores categóricos. Inserimos um ".shape" para obter o tamanho.

In [ ]:
# Esta linha acessa o primeiro lote de dados gerado pela instância e obtém o shape do array numpy contendo as imagens de entrada
sequencia.__getitem__(0)[0].shape

(5, 768, 1024, 3)

# Implementação do modelo CNN

- DepthwiseConv2D:
É uma camada que opera separadamente em cada canal de entrada e foi implementada após ter alguns erros com TensorFlow.
  - kernel_size=(3, 3) define o tamanho do filtro como 3x3;
  - A ativação relu é usada para introduzir não-linearidades no modelo;
  - padding='same' garante que o tamanho da saída seja o mesmo que o da entrada
  - input_shape=(768, 1024, 3) define a forma da entrada do modelo, que neste caso são imagens de 768x1024 pixels com 3 canais de cor (RGB).

- Conv2D(16, kernel_size=(1, 1)):
Essa camada convolucional  reduz a profundidade do mapa de características. Aqui, 16 filtros são aplicados com um tamanho de kernel de 1x1 para reduzir a dimensionalidade entre camadas convolucionais pesadas.

- MaxPooling2D(pool_size=(3, 3)):
Essa camada realiza uma operação de max pooling com uma janela de 3x3, o que reduz a dimensão espacial dos mapas de características pela seleção do valor máximo dentro de cada janela de pooling.

- Conv2D(32, kernel_size=(3, 3)): Uma outra camada convolucional que agora aplica 32 filtros de tamanho 3x3, permitindo ao modelo aprender características mais complexas.

- Flatten():
Esta camada achata os mapas de características multidimensionais em um vetor unidimensional, permitindo que os dados sejam alimentados em camadas totalmente conectadas.

- Dense(60, activation='relu'):
É uma camada totalmente conectada com 60 neurônios

- Dense(14, activation='softmax'): A camada final é uma camada densa com 14 neurônios, cada um correspondendo a uma classe de saída. A função de ativação softmax é usada para calcular a distribuição de probabilidade sobre as 14 classes presentes.

In [ ]:
modelo = Sequential([
    # Adiciona uma camada de convolução em profundidade, que aplica um único filtro por canal de entrada
    DepthwiseConv2D(kernel_size=(3, 3), activation='relu', padding='same', input_shape=(768, 1024, 3)),
    # Adiciona uma camada convolucional que compacta as características aprendidas anteriormente em 16 mapas de características distintos
    Conv2D(16, kernel_size=(1, 1), activation='relu', padding='same'),
    # Adiciona uma camada de pooling máximo com um filtro de 3x3 para reduzir as dimensões espaciais dos mapas
    MaxPooling2D(pool_size=(3, 3)),
    # Adiciona outra camada convolucional para aprender características mais complexas com 32 filtros, mantendo o tamanho da saída igual ao da entrada
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same',),
    # Adiciona outra camada de pooling máximo para continuar a redução dimensional e aumentar a abstração das características
    MaxPooling2D(pool_size=(3, 3)),
    # Achata os mapas de características multidimensionais em um vetor unidimensional para ser possível a entrada em camadas densas
    Flatten(),
    # Adiciona uma camada densa (totalmente conectada)
    Dense(60, activation='relu'),
    # Adiciona a última camada densa com 14 neurônios, um para cada classe.
    Dense(14, activation='softmax')
])

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StatelessRandomUniformV2 in device /job:localhost/replica:0/task:0/device:C

# Efetuando a compilação do modelo

Na compilação atribuímos o tipo de função de perda, otimizador e métricas

- Função de Perda:
A função de perda "categorical_crossentropy" calcula a diferença entre as previsões do modelo e os rótulos no treinamento. Para tarefas de classificação multiclasse, onde cada etiqueta é um vetor de probabilidades(por isso uso da ativação softmax), utiliza-se a entropia cruzada como uma potencial opção. Esta função de perda seve pra penalizar previsões longes dos valores reais


- optimizer=Adam():
A escolha do adam se deve pelo uso semelhante no projeto do grupo, mas também porque o Adam é um otimizador baseado em gradientes que ajusta os pesos da rede durante o treinamento e não precisa de tanta configuração manual dos hiperparâmetros. Ele é capaz de ajustar a taxa de aprendizagem de forma adaptativa para diferentes parâmetros.

- metrics=['accuracy']:
É usada para monitorar o desempenho do modelo durante o treinamento e a validação. Neste caso, a acurácia é a fração de imagens corretamente classificadas em relação ao total de imagens.

In [ ]:
modelo.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device

# Treino do modelo

In [ ]:
#TREINAR MODELO A PARTIR DO OBJETO GERADOR DE SEQUENCIAS

Aqui trata-se do treinamento do modelo. O gerador fornece batches de dados de entrada e saída como imagens ao modelo em cada etapa do treinamento.

 O "epochs" define o número de vezes que o modelo passará pelo conjunto de dados completo. O epochs com valor 5 siginifica que o ciclo de treinamento completo, passando por todos os dados, será repetido cinco vezes.

In [ ]:
modelo.fit(sequencia, epochs= 5)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariable

Após rodar otreinamento do modelo com a CPU, a acurácia obtida foi de 0.98 e a loss foi de 0.06.